In [1]:
import pandas as pd
import numpy as np
import requests
import json
import asyncio 
import aiohttp
import time 

from lxml import html
from datetime import datetime


In [2]:
test = pd.read_csv('test.csv')
test['is_test'] = 1

В ходе анализа test.csv сделал вывод:
Данные только с auto.ru , б/у, оганичены годом от и до.

В ходе аналиаз auto.ru : Данные были вытащены из раметки, 2 - JSON  и чуть из html ...

Парсим данные с auto двумя способами: 
1) Как можно больше
2) Только модели марок тех авто, которые нужно оценить

На каждый способ своя модель, сравним которая лучше.

In [3]:
dfTestCarMod = test['car_url']\
                .apply(lambda x : x.split('/')[6:8] )

dfTestCarMod = pd.DataFrame(\
                            [\
                             dfTestCarMod.apply(lambda x : x[0]),\
                             dfTestCarMod.apply(lambda x : x[1])\
                            ],\
                            index=[0,1]
                           ).T

jsonTestCarMod={}

for brand in dfTestCarMod[0].unique():
    timArr=[]
    timArr = dfTestCarMod[dfTestCarMod[0] == brand][1].unique()
    jsonTestCarMod[brand]=timArr

In [4]:
len(dfTestCarMod)


34686

# Соберем все url по брендам и условиям мин и макс год + перемножим на все сортировки ...

urlFullCar = [];#Все возможные модели марок
urlIfCar = [];# Модели марок только релевантные тестовой 


sort = ['fresh_relevance_1-desc','cr_date-desc','price-asc','price-desc','year-desc','year-asc','km_age-asc','alphabet-asc','autoru_exclusive-desc','price_profitability-desc','proven_owner-desc']


for car in jsonTestCarMod.keys():
    yearMin = test[test['brand']==car].modelDate.min()
    yearMax = test[test['brand']==car].modelDate.max()
    
    carUrl = 'https://auto.ru/cars/{0}/used/?year_from={1}&year_to={2}&sort='.format(car,yearMin,yearMax)
    for so in sort:
        urlFullCar.append(carUrl+so)

        
#Каждую ссылку по условиям выже перемножим на 99 вкладок        
timArr=[]

for url in urlFullCar:
    
    for i in range(1,100):

        timArr.append(url+'&page='+str(i))
        
urlFullCar=timArr


j=1;
jEnd=len(dfTestCarMod[1].unique());

for brand in jsonTestCarMod:
    
    for model in jsonTestCarMod[brand]:
        
        timStr='{0}/{1}'.format(brand,model)
        timStr = 'https://auto.ru/cars/{0}/used/'.format(timStr)
        
        #urlIfCar.append(timStr)
        
        page = requests.get(timStr)
        tree = html.fromstring(page.content)
        timArr = tree.xpath("//a[@class='Button Button_color_whiteHoverBlue Button_size_s Button_type_link Button_width_default ListingPagination__page']/span/span")
        
        if(len (timArr) > 0  ):
            
            for i  in range(1, int( timArr[-1].text )+1):
                urlIfCar.append(timStr+'?page='+str(i))
            
        else:
            urlIfCar.append(timStr)
            
        #print(str(j)+'/'+str(jEnd))
        j+=1

In [5]:
jsonTestCarMod

{'skoda': array(['octavia', 'superb', 'octavia_rs', 'yeti', 'kodiaq', 'rapid',
        'fabia', 'roomster', 'felicia', 'karoq', 'fabia_rs', '120',
        '100_series', 'favorit', 'popular'], dtype=object),
 'audi': array(['q5', 'r8', 'q7', 'q3', 'a4', 'a3', 'a5', 'a6', 'rs7', 's6', 'rs6',
        'allroad', 'tt', 's5', 's8', 'a1', 'a8', '80', 'a7', 'rsq3', 'rs3',
        'a4_allroad', 'q8', '100', 'sq5', 's4', 'tt_rs', 'rs4', 's7',
        'rs5', '90', 'coupe', 'a2', 'tts', '920', 's3', '200', 'v8',
        'e_tron'], dtype=object),
 'honda': array(['cr_v', 'accord', 'pilot', 'civic', 'odyssey', 'stream',
        'crosstour', 'elysion', 'element', 'ridgeline', 'fit', 'hr_v',
        'shuttle', 'freed', 'odyssey_na', 'jazz', 'z', 'civic_ferio',
        's2000', 'cr_z', 'legend', 'stepwagon', 'civic_type_r', 'fr_v',
        'cr_x', 'prelude', 'mobilio_spike', 'n_box', 'zest', 'integra',
        'inspire', 'vezel', 'n_wgn', 'ascot', 'logo', 'acty', 'concerto',
        'avancier', 'saber'

# Генерация urlFullCar_
urlFullCar_ = []
i = 1;
endI = len(urlFullCar)

for url in urlFullCar:

    page = requests.get(url)
    tree = html.fromstring(page.content)
    arr = tree.xpath("//a[@class='Link ListingItemTitle__link']")
    for item in arr:
        urlFullCar_.append(item.get('href'))

    print(str(i)+'/'+str(endI))  
    time.sleep(0.1)
        
    i+=1    
    
urlFullCar = urlFullCar_

pd.DataFrame(pd.Series(urlFullCar).unique()).to_csv('urlFullCar.csv', encoding='utf-8')

In [6]:
urlFullCar = pd.read_csv('urlFullCar.csv')['0']

In [7]:
len(urlFullCar)

96257

urlIfCar_ = []
i = 1;
endI = len(urlIfCar)

for url in urlIfCar:

    page = requests.get(url)
    tree = html.fromstring(page.content)
    arr = tree.xpath("//a[@class='Link ListingItemTitle__link']")
    for item in arr:
        urlIfCar_.append(item.get('href'))

    print(str(i)+'/'+str(endI))
        
    i+=1
urlIfCar = urlIfCar_

pd.DataFrame(pd.Series(urlIfCar).unique()).to_csv('urlIfCar.csv', encoding='utf-8')

In [8]:
urlIfCar = pd.read_csv('urlIfCar_.csv')['0']

In [9]:
urlFullCar

0        https://auto.ru/cars/used/sale/bmw/x5_m/110359...
1        https://auto.ru/cars/used/sale/bmw/x1/11057334...
2        https://auto.ru/cars/used/sale/bmw/7er/1104984...
3        https://auto.ru/cars/used/sale/bmw/7er/1102875...
4        https://auto.ru/cars/used/sale/bmw/5er/1104780...
                               ...                        
96252    https://auto.ru/cars/used/sale/lexus/rx/110582...
96253    https://auto.ru/cars/used/sale/lexus/rx/110600...
96254    https://auto.ru/cars/used/sale/lexus/es/110584...
96255    https://auto.ru/cars/used/sale/lexus/gx/110503...
96256    https://auto.ru/cars/used/sale/lexus/lx/110600...
Name: 0, Length: 96257, dtype: object

In [10]:
urlIfCar

0         https://auto.ru/cars/used/sale/skoda/octavia/1...
1         https://auto.ru/cars/used/sale/skoda/octavia/1...
2         https://auto.ru/cars/used/sale/skoda/octavia/1...
3         https://auto.ru/cars/used/sale/skoda/octavia/1...
4         https://auto.ru/cars/used/sale/skoda/octavia/1...
                                ...                        
106322    https://auto.ru/cars/used/sale/mitsubishi/mini...
106323    https://auto.ru/cars/used/sale/mitsubishi/mini...
106324    https://auto.ru/cars/used/sale/mitsubishi/mini...
106325    https://auto.ru/cars/used/sale/mitsubishi/mini...
106326    https://auto.ru/cars/used/sale/mitsubishi/mini...
Name: 0, Length: 106327, dtype: object

urlIfCar - Ссылка на авто только моделей из выборки для оценки

urlFullCar - Ссылки на авто, все подрят 

In [16]:
from time import time, sleep
import asyncio 
import aiohttp

import requests
import json
from lxml import html

import pandas as pd
import numpy as np

import asyncio

from requests.api import get 

import tracemalloc

tracemalloc.start()


urlFullCar = pd.read_csv('urlFullCar.csv')['0']
urlIfCar = pd.read_csv('urlIfCar_.csv')['0']

print(len(urlFullCar))
print(len(urlIfCar))

async def get_date(url,session):
    obj1, obj2 , arr = await get_contens(url,session)

    if not 'price' in obj1['offers']:
        
        return ['price']

    bodyType = obj1['bodyType']
    brand = obj1['brand']
    car_url = url
    color = obj1['color']
    complectation_dict = obj2['card']['vehicle_info']['complectation']
    description = obj1['description']
    engineDisplacement = obj1['vehicleEngine']['engineDisplacement']
    enginePower = obj1['vehicleEngine']['enginePower'] 
    equipment_dict = obj2['card']['vehicle_info']['equipment']
    fuelType = obj1['fuelType']
    image = obj1['image']
    mileage = obj2['card']['state']['mileage']
    modelDate = obj1['modelDate']
    model_info = obj2['card']['vehicle_info']['model_info']
    model_name =obj2['card']['vehicle_info']['model_info']['code']
    name = obj2['card']['vehicle_info']['tech_param']['human_name']
    numberOfDoors = obj1['numberOfDoors']
    parsing_unixtime = int(time())
    priceCurrency = obj1['offers']['priceCurrency']
    productionDate = obj1['productionDate']
    sell_id = 0
    super_gen = obj2['card']['vehicle_info']['tech_param']
    vehicleConfiguration = obj1['vehicleConfiguration']
    vehicleTransmission = obj1['vehicleTransmission']
    vendor = obj2['card']['vehicle_info']['vendor']
    Владельцы = arr[0]
    Владение = 0# Нет инфы
    ПТС = arr[1]
    Привод = arr[2]
    Руль = arr[3]
    Состояние  = arr[4]
    Таможня  = arr[5]
    price = obj1['offers']['price']

    timArr = [bodyType, brand, car_url, color, complectation_dict,
           description, engineDisplacement, enginePower, str(equipment_dict),
           fuelType, image, mileage, modelDate, model_info, model_name,
           name, numberOfDoors, parsing_unixtime, priceCurrency,
           productionDate, sell_id, super_gen, vehicleConfiguration,
           vehicleTransmission, vendor, Владельцы, Владение, ПТС,
           Привод, Руль, Состояние, Таможня,price]

    return(timArr)



async def get_contens(url,session):
    
    async with session.get(url) as response:

        resText = await response.text()
        tree = html.fromstring(resText)
        obj1 = json.loads( \
                    tree.xpath("//head/script[@type='application/ld+json']")[0].text\
                )

        obj2 = json.loads( \
            tree.xpath("//script[@id='initial-state']")[0].text\
        )    


        Владельцы = tree.xpath("//li[@class = 'CardInfoRow CardInfoRow_ownersCount']/span")[1].text
        ПТС = tree.xpath("//li[@class='CardInfoRow CardInfoRow_pts']/span")[1].text
        Привод = tree.xpath("//li[@class='CardInfoRow CardInfoRow_drive']/span")[1].text
        Руль = tree.xpath("//li[@class='CardInfoRow CardInfoRow_wheel']/span")[1].text
        Состояние  = tree.xpath("//li[@class='CardInfoRow CardInfoRow_state']/span")[1].text
        Таможня  = tree.xpath("//li[@class='CardInfoRow CardInfoRow_customs']/span")[1].text


    return [obj1,obj2,[Владельцы,ПТС,Привод,Руль,Состояние,Таможня]]





async def main(urlArr):
    
    dataArr = []

    tasks = []

    async with aiohttp.ClientSession() as session:
        for url in urlArr:
            task = asyncio.create_task(get_date(url,session))
            tasks.append(task)

        print('stop for!')
        dataArr.append(await asyncio.gather(*tasks))








96257
106327


In [17]:
if __name__ == '__main__':
    t0 = time()
    asyncio.run(main(urlIfCar))
    print(time()-t0)


RuntimeError: asyncio.run() cannot be called from a running event loop

In [22]:
def ferch_date(url):   
  
    page = requests.get(url)
    tree = html.fromstring(page.content)

    jsDataOne = json.loads( \
                       tree.xpath("//head/script[@type='application/ld+json']")[0].text\
                      )
    jsDataTwo = json.loads( \
               tree.xpath("//script[@id='initial-state']")[0].text\
              )

    if not 'price' in jsDataOne['offers']:
        
        return []

    bodyType = jsDataOne['bodyType']
    brand = jsDataOne['brand']
    car_url = url
    color = jsDataOne['color']
    complectation_dict = jsDataTwo['card']['vehicle_info']['complectation']
    description = jsDataOne['description']
    engineDisplacement = jsDataOne['vehicleEngine']['engineDisplacement']
    enginePower = jsDataOne['vehicleEngine']['enginePower'] 
    equipment_dict = jsDataTwo['card']['vehicle_info']['equipment']
    fuelType = jsDataOne['fuelType']
    image = jsDataOne['image']
    mileage = jsDataTwo['card']['state']['mileage']
    modelDate = jsDataOne['modelDate']
    model_info = jsDataTwo['card']['vehicle_info']['model_info']
    model_name =jsDataTwo['card']['vehicle_info']['model_info']['code']
    name = jsDataTwo['card']['vehicle_info']['tech_param']['human_name']
    numberOfDoors = jsDataOne['numberOfDoors']
    parsing_unixtime = int(time.mktime(datetime.now().timetuple()))
    priceCurrency = jsDataOne['offers']['priceCurrency']
    productionDate = jsDataOne['productionDate']
    sell_id = 0
    super_gen = jsDataTwo['card']['vehicle_info']['tech_param']
    vehicleConfiguration = jsDataOne['vehicleConfiguration']
    vehicleTransmission = jsDataOne['vehicleTransmission']
    vendor = jsDataTwo['card']['vehicle_info']['vendor']
    Владельцы = tree.xpath("//li[@class = 'CardInfoRow CardInfoRow_ownersCount']/span")[1].text
    Владение = 0# Нет инфы
    ПТС = tree.xpath("//li[@class='CardInfoRow CardInfoRow_pts']/span")[1].text
    Привод = tree.xpath("//li[@class='CardInfoRow CardInfoRow_drive']/span")[1].text
    Руль = tree.xpath("//li[@class='CardInfoRow CardInfoRow_wheel']/span")[1].text
    Состояние  = tree.xpath("//li[@class='CardInfoRow CardInfoRow_state']/span")[1].text
    Таможня  = tree.xpath("//li[@class='CardInfoRow CardInfoRow_customs']/span")[1].text
    price = jsDataOne['offers']['price']

    timArr = [bodyType, brand, car_url, color, complectation_dict,
           description, engineDisplacement, enginePower, str(equipment_dict),
           fuelType, image, mileage, modelDate, model_info, model_name,
           name, numberOfDoors, parsing_unixtime, priceCurrency,
           productionDate, sell_id, super_gen, vehicleConfiguration,
           vehicleTransmission, vendor, Владельцы, Владение, ПТС,
           Привод, Руль, Состояние, Таможня,price]

    #timData=pd.DataFrame(np.array([timArr]),\
    #                    columns=['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
    #       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
    #       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
    #       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
    #       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
    #       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
    #       'Привод', 'Руль', 'Состояние', 'Таможня','price'])
    
    return(timArr)

In [11]:
urlIfCar[0]

'https://auto.ru/cars/used/sale/skoda/octavia/1105878245-472c3ac4/'

In [28]:
from multiprocessing import Pool

def f(x):
    return x*x
    

start = time.time()
p = Pool(6)
dataUrlIfCar = p.map(funcGetParam, urlIfCar)

print('Время {0} sec'.format(-1*start + time.time()))

IndexError: list index out of range

In [29]:
len(dataUrlIfCar)

1000

In [32]:
start = time()

In [46]:
(-1*start + time())

56.303168058395386

In [26]:
len(dataUrlIfCar)

1000

In [33]:
start

1637585483.3487368

In [7]:
arrFullCar = []

In [11]:
for i in range(0,20):
    arrFullCar.append(pd.read_csv('train_{}.csv'.format(i)))

In [14]:
arrFullCar

[      Unnamed: 0                                                  0
 0              0  ['внедорожник 5 дв.', 'BMW', 'https://auto.ru/...
 1              1  ['внедорожник 5 дв.', 'BMW', 'https://auto.ru/...
 2              2  ['седан', 'BMW', 'https://auto.ru/cars/used/sa...
 3              3  ['седан', 'BMW', 'https://auto.ru/cars/used/sa...
 4              4                                          ['price']
 ...          ...                                                ...
 4995        4995  ['седан', 'BMW', 'https://auto.ru/cars/used/sa...
 4996        4996  ['седан', 'BMW', 'https://auto.ru/cars/used/sa...
 4997        4997  ['седан', 'BMW', 'https://auto.ru/cars/used/sa...
 4998        4998                                          ['price']
 4999        4999  ['седан', 'BMW', 'https://auto.ru/cars/used/sa...
 
 [5000 rows x 2 columns],
       Unnamed: 0                                                  0
 0              0                                          ['price']
 1    

In [24]:
trainFull = pd.DataFrame(np.concatenate(arrFullCar))[1]

In [26]:
trainFull.to_csv('trainFull.csv')

In [30]:
arrIfCat = []

In [31]:
for i in range(0,22):
    arrIfCat.append(pd.read_csv('train_if_{}.csv'.format(i)))

In [32]:
trainIf = pd.DataFrame(np.concatenate(arrIfCat))[1]

In [34]:
trainIf.to_csv('trainIf.csv')